In [1]:
from langchain_community.tools import ArxivQueryRun,WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper,ArxivAPIWrapper

In [4]:
api_wrapper_wiki = WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=250)
wiki=WikipediaQueryRun(api_wrapper=api_wrapper_wiki)
wiki.name

'wikipedia'

In [5]:
arxiv_wrapper_wiki = ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=250)
arxiv=ArxivQueryRun(api_wrapper=api_wrapper_wiki)
arxiv.name

'arxiv'

In [6]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["HF_TOKEN"]=os.getenv("HF_TOKEN")

In [9]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-l6-v2")

/Users/joodithhermminal/projects/ai_softwares/langchain_components/venv/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [11]:
loader=WebBaseLoader("https://docs.smith.langchain.com")
docs=loader.load()
splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
split_docs=splitter.split_documents(docs)
vectordb=FAISS.from_documents(split_docs,embeddings)
retriever=vectordb.as_retriever()
retriever


VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x1776f9690>)

In [12]:
from langchain.tools.retriever import create_retriever_tool
retrieval_tool=create_retriever_tool(retriever,"Langsmith search","Search any information about langsmith")



In [19]:
tools=[arxiv,wiki,retrieval_tool]


In [20]:
os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")

from langchain_groq import ChatGroq

llm= ChatGroq(model="llama3-8b-8192")

In [23]:
from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/react")
prompt

/Users/joodithhermminal/projects/ai_softwares/langchain_components/venv/lib/python3.11/site-packages/langchain_core/_api/beta_decorator.py:87: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  warn_beta(


PromptTemplate(input_variables=['agent_scratchpad', 'input', 'tool_names', 'tools'], metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'react', 'lc_hub_commit_hash': 'd15fe3c426f1c4b3f37c9198853e4a86e20c425ca7f4752ec0c9b0e97ca7ea4d'}, template='Answer the following questions as best you can. You have access to the following tools:\n\n{tools}\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [{tool_names}]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: {input}\nThought:{agent_scratchpad}')

In [26]:
agent= create_react_agent(llm, tools, prompt)
agent_executor=AgentExecutor(agent=agent, tools=tools, verbose=True)

In [27]:
agent_executor.invoke({"input":"Explain what is the use of langsmith in less technical terms"})



> Entering new AgentExecutor chain...
Thought: Langsmith seems to be a tool that helps search and retrieve information from various sources, including academic papers, Wikipedia, and more. I think I can use it to explain what Langsmith is and what it can do.

Action: Langsmith search

Action Input: 'What is Langsmith?'
Get started with LangSmith | 🦜️🛠️ LangSmith

Log your first trace​Tracing to LangSmith for LangChain usersThere is no need to use the LangSmith SDK directly if your application is built entirely on LangChain (either Python and JS).We've outlined a tracing guide specifically for LangChain users here.We provide multiple ways to log traces to LangSmith. Below, we'll highlight

Skip to main contentGo to API DocsSearchRegionUSEUGo to AppQuick startTutorialsHow-to guidesConceptsReferencePricingSelf-hostingLangGraph CloudQuick startOn this pageGet started with LangSmithLangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and

{'input': 'Explain what is the use of langsmith in less technical terms',
 'output': 'The final answer is Langsmith is a platform for building production-grade LLM applications, and it allows developers to closely monitor and evaluate their applications so they can ship quickly and with confidence. It has features such as tracing, which enables developers to log and analyze the behavior of their applications. Langsmith also provides a tracing guide specifically for LangChain users, and it can be used with OpenAI API.'}

In [28]:
agent_executor.invoke({"input":"What is deep learning and how does it work?"})



> Entering new AgentExecutor chain...
Thought: This question seems to be related to computer science and artificial intelligence. I should try to find a scientific article on arXiv that explains deep learning and its working mechanism.

Action: arxiv

Action Input: deep learning
No good Arxiv Result was foundThought: Since I couldn't find a suitable article on ArXiv, I should try to find a more general explanation of deep learning on Wikipedia.

Action: wikipedia

Action Input: deep learning
Page: Deep learning
Summary: Deep learning is a subset of machine learning methods based on neural networks with representation learning. The field takes inspiration from biological neuroscience and is centered around stacking artificial neurons intoHere's the continuation of the conversation:

Thought: Thought: The Wikipedia summary provides a good general overview of deep learning, but I'd like to get more technical details. I'll try using Langsmith to search for a more detailed explanation.

A

{'input': 'What is deep learning and how does it work?',
 'output': 'Deep learning is a subset of machine learning methods based on neural networks with representation learning. The field takes inspiration from biological neuroscience and is centered around stacking artificial neurons into hierarchical layers.'}